# Library 

In [1]:
import numpy as np
from selenium import webdriver
from time import sleep
import re
from pymongo import MongoClient
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import random
from selenium.common.exceptions import NoSuchElementException , ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options

# Class Crawl

In [2]:
class batdongsan():
    def __init__(self,link,name=[]):
        self.link = link
        self.name = name
        self.result = pd.DataFrame(columns=name)
        options= Options()
        options.page_load_strategy = 'eager'
        options.headless = True
        options.add_argument(r"--user-data-dir=C:\\Users\\USER\AppData\\Local\\Google\\Chrome\\User Data\Default")
        options.add_argument("--disable-javascript")
        options.add_argument("--blink-settings=imagesEnabled=false")
        self.driver = webdriver.Chrome(options=options)
    def Crawl(self):
        web_page=self.link
        while True :
            List_link , next_page = self.get_links(web_page)
            for i in range(len(List_link)-1):
                # if List_link[i] in link_have :
                #     continue
                self.getData(List_link[i])
            web_page = next_page
            if web_page == None:
                break
    def get_links(self,link):
        driver = self.driver
        driver.get(link)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
            'a.js__product-link-for-product-id'))
        )

        text = driver.find_elements(By.CSS_SELECTOR,
        'a.js__product-link-for-product-id')
        links= [i.get_attribute('href') for i in text]

        next_page = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR,
            'a.re__pagination-icon'))
        )
        next_page = next_page.get_attribute('href')
        
        return links  ,next_page
    def getData(self,link):
        driver = self.driver
        test = pd.Series(index=self.name)
        driver.get(link)


        # Các nội dung tổng quan về mẫu
        test['Link'] = link
        test['Title'] = driver.find_element(By.CSS_SELECTOR,'h1.re__pr-title').text
        test['Address'] = driver.find_element(By.CSS_SELECTOR,'.js__pr-address').text

        # Thử lấy dữ liệu xác thực và số zalo liên hệ
        try:
            test['Check'] = driver.find_element(By.CSS_SELECTOR,
            '.js__product-detail-web .re__pr-stick-listing-verified .re__text').text
            
        except NoSuchElementException :
            test['Check']='Chưa Xác Thực'
        # Thử lấy dữ liệu chi tiêt về mẫu
        name_data = driver.find_elements(By.CSS_SELECTOR,

        '.js__li-specs  .js__section-body .js__other-info .re__pr-specs-content-item .re__pr-specs-content-item-title')
        value_data =  driver.find_elements(By.CSS_SELECTOR,

        '.js__li-specs  .js__section-body .js__other-info .re__pr-specs-content-item .re__pr-specs-content-item-value')
        data = pd.Series(value_data,index=name_data)

        for i in range(len(name_data)):
            test[name_data[i].text] = value_data[i].text

        name_data_2 = driver.find_elements(By.CSS_SELECTOR,
        '.js__pr-config .js__pr-config-item .title')

        value_data_2 = driver.find_elements(By.CSS_SELECTOR,
        '.js__pr-config .js__pr-config-item .value')

        for i in range(len(name_data_2)):
            test[name_data_2[i].text] = value_data_2[i].text

        self.result = pd.concat([self.result, test.to_frame().T], ignore_index=True)


# Crawl Data

- Infomation before crawl

In [3]:
nameCol = ['Link','Diện tích', 'Mức giá','Hướng nhà', 'Số tầng', 'Số phòng ngủ', 'Số toilet', 'Pháp lý',
 'Nội thất','Title','Address','Ngày đăng','Ngày hết hạn','Loại tin','Mã tin','Check']
link_web = 'https://batdongsan.com.vn/ban-nha-dat-tp-hcm'

- List sample that was colected to not crawl dulicate sample

In [4]:
# df_have = pd.read_csv('E:\DE_JOB\Presonal_Project\Pineline_ETL\CrawlData\\batdongsan\\batdongsan.csv')
# link_have=df_have['Link']

- Create Class

In [5]:
X = batdongsan(link=link_web,name=nameCol)

- Crawl Data

In [6]:
X.Crawl()

C:\Users\USER\AppData\Local\Temp\ipykernel_14212\850186965.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://batdongsan.com.vn/ban-nha-rieng-duong-pham-ngu-lao-phuong-3-20/ban-dep-khu-pho-an-ninh-41-12-p-3-go-vap-pr41422117' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test['Link'] = link
C:\Users\USER\AppData\Local\Temp\ipykernel_14212\850186965.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://batdongsan.com.vn/ban-nha-rieng-duong-hoang-van-thu-phuong-9-12/ban-32m2-2-mat-tien-tai-phu-nhuan-gia-6-85-ty-4-pn-3-pt-pr41427152' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test['Link'] = link
C:\Users\USER\AppData\Local\Temp\ipykernel_14212\850186965.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will rais

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"h1.re__pr-title"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6D4C438A5+3004357]
	(No symbol) [0x00007FF6D48D9970]
	(No symbol) [0x00007FF6D478582A]
	(No symbol) [0x00007FF6D47D5B8E]
	(No symbol) [0x00007FF6D47D5E7C]
	(No symbol) [0x00007FF6D481EC27]
	(No symbol) [0x00007FF6D47FBC1F]
	(No symbol) [0x00007FF6D481BA4C]
	(No symbol) [0x00007FF6D47FB983]
	(No symbol) [0x00007FF6D47C7628]
	(No symbol) [0x00007FF6D47C8791]
	GetHandleVerifier [0x00007FF6D4C6A00D+3161901]
	GetHandleVerifier [0x00007FF6D4CBE060+3506048]
	GetHandleVerifier [0x00007FF6D4CB400D+3465005]
	GetHandleVerifier [0x00007FF6D4A30EEB+830987]
	(No symbol) [0x00007FF6D48E467F]
	(No symbol) [0x00007FF6D48E09D4]
	(No symbol) [0x00007FF6D

# Store Data

- MongoDB

In [7]:
client = MongoClient("mongodb://localhost:27017/")
db = client["batdongsan"]
collection = db["batdongsan"]
data=X.result
data_dict = data.to_dict("records")
collection.insert_many(data_dict)

InsertManyResult([ObjectId('672c2a5d116e250d3d8b8e1c'), ObjectId('672c2a5d116e250d3d8b8e1d'), ObjectId('672c2a5d116e250d3d8b8e1e'), ObjectId('672c2a5d116e250d3d8b8e1f'), ObjectId('672c2a5d116e250d3d8b8e20'), ObjectId('672c2a5d116e250d3d8b8e21'), ObjectId('672c2a5d116e250d3d8b8e22'), ObjectId('672c2a5d116e250d3d8b8e23'), ObjectId('672c2a5d116e250d3d8b8e24'), ObjectId('672c2a5d116e250d3d8b8e25'), ObjectId('672c2a5d116e250d3d8b8e26'), ObjectId('672c2a5d116e250d3d8b8e27'), ObjectId('672c2a5d116e250d3d8b8e28'), ObjectId('672c2a5d116e250d3d8b8e29'), ObjectId('672c2a5d116e250d3d8b8e2a'), ObjectId('672c2a5d116e250d3d8b8e2b')], acknowledged=True)

- PostgresSQL

In [ ]:
from sqlalchemy import create_engine
data = X.result
# Thông tin kết nối PostgreSQL
db_user = "postgres"
db_password = "ky1213141516"
db_host = "localhost"
db_port = "3000"
db_name = "postgres"

# Tạo kết nối với PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Lưu DataFrame vào PostgreSQL (chọn tên bảng phù hợp)
data.to_sql('batdongsan', engine, if_exists='replace', index=False)

23